# Traffic Vision: Multi-Camera Recording System

## Objective
Develop a Linux system-level service for continuous multi-camera GDOT traffic stream recording.

## Technical Approach
- **System Level:** Bash scripts + systemd service for 24/7 operation
- **Recording Method:** Streamlink + FFmpeg (Method 3 from validation tests)
- **Data Source:** CSV file with camera definitions
- **Storage Structure:** `recordings/CAMERA-ID/YYYY-MM-DD/CAMERA-ID_YYYYMMDD_HHMMSS.mp4`

## Recording Configuration
- **Development/Testing:** 15-second segments (for workflow demo, included in repo)
- **Production Deployment:** 15-minute segments, 24/7 continuous on bare metal storage

## Current Implementation
- **Base Script:** `multi_camera_recorder.sh` (reads CSV, spawns parallel processes)
- **Target:** 8 Atlanta midtown cameras simultaneous recording

## Next Steps
1. Validate CSV data loading and stream accessibility
2. Test parallel recording with 2 cameras
3. Scale to full 8-camera deployment
4. Configure systemd service for continuous operation

**Date:** June 8, 2025  
**Branch:** `experiment/multi-camera-recording`

In [4]:
import pandas as pd
import os

import os
print("current dir:", os.getcwd())
print("files here:", os.listdir('.'))

# load csv data
csv_file = '/home/trauco/traffic-vision/data/gdot_midtown_atlanta_streams.csv'
df = pd.read_csv(csv_file)

print(f"dataset: {len(df)} cameras")
print(f"columns: {list(df.columns)}")
print("\nfirst cameras:")
print(df.head())

current dir: /home/trauco/traffic-vision/notebooks
files here: ['.ipynb_checkpoints', 'traffic_vision_multi_camera_recording_experiment.ipynb']
dataset: 8 cameras
columns: ['Date Added', 'Camera_ID', 'Location', 'Page', 'Stream_URL']

first cameras:
   Date Added Camera_ID                               Location  Page  \
0  2025-04-11  ATL-0610         10th St at Monroe Dr (Atlanta)     6   
1  2025-04-11  ATL-0972       Peachtree St at 5th St (Atlanta)    15   
2  2025-04-11  ATL-1005      Peachtree St at 12th St (Atlanta)    15   
3  2025-04-11  ATL-0907      Piedmont Ave at 14th St (Atlanta)    15   
4  2025-04-11  ATL-0997  West Peachtree St at 5th St (Atlanta)    27   

                                          Stream_URL  
0  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
1  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
2  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
3  https://sfs-msc-pub-lq-01.navigator.dot.ga.gov...  
4  https://sfs-msc-pub-lq-01.navigator.dot

In [8]:
import subprocess
import json
import glob

# analyze recent recordings
recording_files = glob.glob('/home/trauco/traffic-vision/recordings/*/*.mp4')

print(f"found {len(recording_files)} recordings")

for file in recording_files[:3]:  # check first 3
    print(f"\n{file}:")
    
    # get file size
    size = os.path.getsize(file)
    print(f"  size: {size:,} bytes ({size/1024/1024:.1f}MB)")
    
    # get video metadata with ffprobe
    cmd = ['ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', '-show_streams', file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        data = json.loads(result.stdout)
        duration = float(data['format']['duration'])
        video_stream = next((s for s in data['streams'] if s['codec_type'] == 'video'), None)
        
        if video_stream:
            fps = eval(video_stream['r_frame_rate'])
            width = video_stream['width']
            height = video_stream['height']
            
            print(f"  duration: {duration:.1f}s")
            print(f"  resolution: {width}x{height}")
            print(f"  fps: {fps:.1f}")
            print(f"  codec: {video_stream['codec_name']}")

found 4 recordings

/home/trauco/traffic-vision/recordings/ATL-0972/ATL-0972_20250608_201300.mp4:
  size: 100,749 bytes (0.1MB)
  duration: 16.5s
  resolution: 480x270
  fps: 15.0
  codec: h264

/home/trauco/traffic-vision/recordings/ATL-0972/ATL-0972_20250608_202414.mp4:
  size: 129,784 bytes (0.1MB)
  duration: 16.5s
  resolution: 480x270
  fps: 15.0
  codec: h264

/home/trauco/traffic-vision/recordings/ATL-0610/ATL-0610_20250608_201300.mp4:
  size: 248,641 bytes (0.2MB)
  duration: 20.7s
  resolution: 480x270
  fps: 15.0
  codec: h264


## Recording Quality Analysis

Test recordings from 2 cameras:

**ATL-0972:**
- 100,749 bytes (0.1MB), 16.5s, 480x270, 15.0 fps, h264
- 129,784 bytes (0.1MB), 16.5s, 480x270, 15.0 fps, h264

**ATL-0610:**
- 248,641 bytes (0.2MB), 20.7s, 480x270, 15.0 fps, h264

All recordings: 480x270 resolution, 15 fps, h264 codec